In [1]:
import duckdb
import leafmap
import pandas as pd

In [2]:
con = duckdb.connect()
con.install_extension("httpfs")
con.load_extension("httpfs")
con.install_extension("spatial")
con.load_extension("spatial")

In [3]:
# world connnectedness data from https://data.humdata.org/dataset/social-connectedness-index
url = 'https://data.humdata.org/dataset/e9988552-74e4-4ff4-943f-c782ac8bca87/resource/35ca6ade-a5bd-4782-b266-797169dca74b/download/countries-countries-fb-social-connectedness-index-october-2021.tsv'

In [4]:
# Read the data from the URL into a pandas DataFrame
df = pd.read_csv(url, sep='\t')

# Convert the DataFrame into a DuckDB table
con.register('my_table', df)

# Now you can query the table using DuckDB
result = con.execute("SELECT * FROM my_table").fetchdf()

In [5]:
df

,user_loc,fr_loc,scaled_sci
0,AE,AE,1117179
1,AE,AG,5488
2,AE,AL,2101
3,AE,AM,3580
4,AE,AO,2326
...,...,...,...
34220,ZW,XK,262
34221,ZW,YT,1650
34222,ZW,ZA,366466
34223,ZW,ZM,196035


In [6]:
con.sql('''
    SELECT user_loc as country, SUM(scaled_sci) as scaled
    FROM my_table
    GROUP BY user_loc
    ORDER BY scaled DESC
    LIMIT 10            
''')

┌─────────┬────────────┐
│ country │   scaled   │
│ varchar │   int128   │
├─────────┼────────────┤
│ SC      │ 1006368457 │
│ AG      │  893365845 │
│ GD      │  835369954 │
│ KI      │  782782835 │
│ TO      │  719577449 │
│ ST      │  684169236 │
│ VC      │  667785101 │
│ FM      │  619429818 │
│ LC      │  578010894 │
│ IM      │  514740394 │
├─────────┴────────────┤
│ 10 rows    2 columns │
└──────────────────────┘

In [7]:
# SC Seychelles, AG Antigua/Barbuda, GD Grenada, KI Kiribati, TO Tonga, ST Sao Tome, VC St Vincent, FM Micronesia, LC St. Lucia, IM Isle of Man

In [8]:
# US counties data from https://data.humdata.org/dataset/social-connectedness-index
url_state = 'https://data.humdata.org/dataset/e9988552-74e4-4ff4-943f-c782ac8bca87/resource/868a2fdb-f5c8-4a98-af7c-cfc8bf0daeb3/download/us-counties-countries-fb-social-connectedness-index-october-2021.tsv'

In [9]:
# Read the data from the URL into a pandas DataFrame
df_zip = pd.read_csv(url_state, sep='\t')

# Register the DataFrame as a DuckDB table
con.register('state_table', df_zip)

# Perform the grouping and ordering operations
result = con.execute("""
    SELECT user_loc as ZIP, SUM(scaled_sci) as SCALED
    FROM state_table
    GROUP BY user_loc
    ORDER BY SCALED DESC
    LIMIT 10
""").fetchdf()
result

,ZIP,SCALED
0,66010,1.235691e+09
1,69100,6.259993e+08
2,60010,4.407371e+08
3,69110,2.921051e+08
4,72107,2.793023e+08
5,72019,2.662062e+08
6,72105,2.578541e+08
7,72073,2.571212e+08
8,78010,2.555521e+08
9,72149,2.472858e+08


In [10]:
# Import the UC Woody Biomass Utilization Group's Current Wood Facility Databases
# Requested online, received email with zipped folders
# Start with sawmills
con.sql("SELECT * FROM ST_Read('C:/Users/vance/Downloads/CurrentSawmill/Current_Wood_Facility_Database_Primary_Wood_Processing.shp')")

┌───────┬─────────────┬──────────────┬───┬────────────┬──────────────────────┬──────────┬──────────────────────┐
│ RecID │  latitude   │  longitude   │ … │    Year    │      JoinSymbol      │ ObjectId │         geom         │
│ int32 │   double    │    double    │   │    date    │       varchar        │  int32   │       geometry       │
├───────┼─────────────┼──────────────┼───┼────────────┼──────────────────────┼──────────┼──────────────────────┤
│  9001 │ 40.79719014 │ -124.1815646 │ … │ 2020-01-01 │ Operational Large …  │       30 │ POINT (-124.181564…  │
│  9002 │ 38.90448023 │ -121.3070281 │ … │ 2020-01-01 │ Operational Large …  │       31 │ POINT (-121.307028…  │
│  9003 │ 39.48193681 │ -121.5632263 │ … │ 2020-01-01 │ Operational Cedar …  │       32 │ POINT (-121.563226…  │
│  9008 │   40.471498 │  -122.320991 │ … │ 2020-01-01 │ Operational Large …  │       33 │ POINT (-122.320991…  │
│  9012 │   40.901169 │  -124.071143 │ … │ 2020-01-01 │ Operational Fence …  │       34 │ POINT 

In [11]:
# Create sawmill table from shp file and show
con.sql('''
    CREATE TABLE IF NOT EXISTS sawmill as
    SELECT * FROM ST_Read('C:/Users/vance/Downloads/CurrentSawmill/Current_Wood_Facility_Database_Primary_Wood_Processing.shp')    
''')
con.table('sawmill')

┌───────┬─────────────┬──────────────┬───┬────────────┬──────────────────────┬──────────┬──────────────────────┐
│ RecID │  latitude   │  longitude   │ … │    Year    │      JoinSymbol      │ ObjectId │         geom         │
│ int32 │   double    │    double    │   │    date    │       varchar        │  int32   │       geometry       │
├───────┼─────────────┼──────────────┼───┼────────────┼──────────────────────┼──────────┼──────────────────────┤
│  9001 │ 40.79719014 │ -124.1815646 │ … │ 2020-01-01 │ Operational Large …  │       30 │ POINT (-124.181564…  │
│  9002 │ 38.90448023 │ -121.3070281 │ … │ 2020-01-01 │ Operational Large …  │       31 │ POINT (-121.307028…  │
│  9003 │ 39.48193681 │ -121.5632263 │ … │ 2020-01-01 │ Operational Cedar …  │       32 │ POINT (-121.563226…  │
│  9008 │   40.471498 │  -122.320991 │ … │ 2020-01-01 │ Operational Large …  │       33 │ POINT (-122.320991…  │
│  9012 │   40.901169 │  -124.071143 │ … │ 2020-01-01 │ Operational Fence …  │       34 │ POINT 

In [12]:
# View sawmill table schema
con.sql('''
    DESCRIBE sawmill
        
''')

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│ column_name │ column_type │  null   │   key   │ default │ extra │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ int32 │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ RecID       │ INTEGER     │ YES     │ NULL    │ NULL    │  NULL │
│ latitude    │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ longitude   │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ Name        │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ Status      │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ Owner       │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ Cogenerati  │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ Facility_T  │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ Feedstock_  │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ Employees   │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ StAddr      │ VARCHAR     │ YES     │ NULL    

In [13]:
# Sum sawmills by county
con.sql('''
    SELECT County, COUNT(*) as Count
    FROM sawmill
    GROUP BY County
    ORDER BY count DESC
''')

┌────────────┬───────┐
│   County   │ Count │
│  varchar   │ int64 │
├────────────┼───────┤
│ Shasta     │     5 │
│ Humboldt   │     4 │
│ Sonoma     │     3 │
│ Plumas     │     2 │
│ Butte      │     2 │
│ Mendocino  │     2 │
│ Siskiyou   │     2 │
│ Tuolumne   │     2 │
│ Trinity    │     1 │
│ Lake, OR   │     1 │
│ Placer     │     1 │
│ Tulare     │     1 │
│ Santa Cruz │     1 │
├────────────┴───────┤
│ 13 rows  2 columns │
└────────────────────┘

In [14]:
# Import the Biomass dataset
con.sql("SELECT * FROM ST_Read('C:/Users/vance/Downloads/CurrentBiomass/Current_Wood_Facility_Database_Biomass.shp')")

┌───────┬─────────────┬──────────────┬──────────────────────┬───┬────────────┬──────────┬──────────────────────┐
│ RecID │  latitude   │  longitude   │         Name         │ … │ JoinSymbol │ ObjectId │         geom         │
│ int32 │   double    │    double    │       varchar        │   │  varchar   │  int32   │       geometry       │
├───────┼─────────────┼──────────────┼──────────────────────┼───┼────────────┼──────────┼──────────────────────┤
│    25 │   35.576448 │  -119.005818 │ DTE Mt. Poso Cogen   │ … │ 25_2021    │       27 │ POINT (-119.005818…  │
│    14 │   36.569581 │  -119.418984 │ Dinuba Energy        │ … │ 14_2021    │       56 │ POINT (-119.418984…  │
│    15 │   38.524353 │  -121.903077 │ Dixon Ridge Farms …  │ … │ 15_2021    │       94 │ POINT (-121.903077…  │
│    16 │   37.943945 │  -121.330053 │ DTE Stockton Bioma…  │ … │ 16_2021    │      110 │ POINT (-121.330053…  │
│    22 │   36.755903 │  -120.365273 │ Covanta Mendota Po…  │ … │ 22_2021    │      122 │ POINT 

In [15]:
# Create biomass table from shp file and show
con.sql('''
    CREATE TABLE IF NOT EXISTS biomass as
    SELECT * FROM ST_Read('C:/Users/vance/Downloads/CurrentBiomass/Current_Wood_Facility_Database_Biomass.shp')    
''')
con.table('sawmill')

┌───────┬─────────────┬──────────────┬───┬────────────┬──────────────────────┬──────────┬──────────────────────┐
│ RecID │  latitude   │  longitude   │ … │    Year    │      JoinSymbol      │ ObjectId │         geom         │
│ int32 │   double    │    double    │   │    date    │       varchar        │  int32   │       geometry       │
├───────┼─────────────┼──────────────┼───┼────────────┼──────────────────────┼──────────┼──────────────────────┤
│  9001 │ 40.79719014 │ -124.1815646 │ … │ 2020-01-01 │ Operational Large …  │       30 │ POINT (-124.181564…  │
│  9002 │ 38.90448023 │ -121.3070281 │ … │ 2020-01-01 │ Operational Large …  │       31 │ POINT (-121.307028…  │
│  9003 │ 39.48193681 │ -121.5632263 │ … │ 2020-01-01 │ Operational Cedar …  │       32 │ POINT (-121.563226…  │
│  9008 │   40.471498 │  -122.320991 │ … │ 2020-01-01 │ Operational Large …  │       33 │ POINT (-122.320991…  │
│  9012 │   40.901169 │  -124.071143 │ … │ 2020-01-01 │ Operational Fence …  │       34 │ POINT 

In [16]:
# View biomass table schema
con.sql('''
    DESCRIBE biomass
        
''')

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│ column_name │ column_type │  null   │   key   │ default │ extra │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ int32 │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ RecID       │ INTEGER     │ YES     │ NULL    │ NULL    │  NULL │
│ latitude    │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ longitude   │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ Name        │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ Status      │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ Owner       │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ Cogenerati  │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ Facility_T  │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ Feedstock_  │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ MW_Namepla  │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ MW_Grid     │ VARCHAR     │ YES     │ NULL    

In [17]:
# Sum the total MW_Grid by county round to 1 decimal

con.sql('''
    SELECT County, ROUND(SUM(CAST(MW_Grid AS FLOAT)), 1) as Total_MW_Grid
    FROM biomass
    GROUP BY County
    ORDER BY Total_MW_Grid DESC
    LIMIT 15
''')

┌─────────────┬───────────────┐
│   County    │ Total_MW_Grid │
│   varchar   │    double     │
├─────────────┼───────────────┤
│ Shasta      │          94.0 │
│ San Joaquin │          45.0 │
│ Riverside   │          44.0 │
│ Kern        │          44.0 │
│ Placer      │          35.4 │
│ Jackson     │          30.0 │
│ Humboldt    │          27.8 │
│ Fresno      │          24.3 │
│ Lassen      │          24.0 │
│ Tuolumne    │          20.0 │
│ Plumas      │          16.0 │
│ Sierra      │          13.0 │
│ Siskiyou    │          12.0 │
│ Madera      │          10.0 │
│ Yolo        │           0.0 │
├─────────────┴───────────────┤
│ 15 rows           2 columns │
└─────────────────────────────┘

In [18]:
# Add h3 schema or show both data sets on map


In [19]:
con.sql('''
    CREATE TABLE IF NOT EXISTS fires as
    SELECT * FROM ST_Read('cafires.csv')    
''')
con.table('fires')

┌─────────┬─────────┬─────────┬────────────┬─────────┬───┬─────────┬─────────┬──────────┬─────────────┬───────────────┐
│  UNIT   │  YEAR   │ DECADE  │   ACRES    │ AGENCY  │ … │  DAYS   │  CAUSE  │ OBJECTID │ Shape__Area │ Shape__Length │
│ varchar │ varchar │ varchar │  varchar   │ varchar │   │ varchar │ varchar │ varchar  │   varchar   │    varchar    │
├─────────┼─────────┼─────────┼────────────┼─────────┼───┼─────────┼─────────┼──────────┼─────────────┼───────────────┤
│ MNF     │ 2020    │ 2020    │ 1032699.63 │ USF     │ … │ 87      │ 1       │ 21743    │ 7116307900  │ 1695542.555   │
│ BTU     │ 2021    │ 2020    │ 963405.44  │ CDF     │ … │ 103     │ 11      │ 43182    │ 6692866165  │ 2010262.448   │
│ SIF     │ 2002    │ 2000    │ 501082.03  │ USF     │ … │ 118     │ 1       │ 32139    │ 3708702192  │ 545191.7425   │
│ LNU     │ 2018    │ 2010    │ 410202.47  │ CDF     │ … │ 54      │ 9       │ 42292    │ 2774859061  │ 450086.7901   │
│ SCU     │ 2020    │ 2020    │ 396824.4

In [20]:
con.sql('''
    DESCRIBE fires
''')

┌───────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│  column_name  │ column_type │  null   │   key   │ default │ extra │
│    varchar    │   varchar   │ varchar │ varchar │ varchar │ int32 │
├───────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ UNIT          │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ YEAR          │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ DECADE        │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ ACRES         │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ AGENCY        │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ NAME          │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ START         │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ CONTAINED     │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ DAYS          │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ CAUSE         │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ OBJECTID      │ VA

In [21]:
# Add a new column 'ACRES_FLOAT' to the 'fires' table
con.sql('''
    ALTER TABLE fires
    ADD COLUMN ACRES_FLOAT FLOAT
''')

# Update the 'ACRES_FLOAT' column with the 'ACRES' values converted to float
con.sql('''
    UPDATE fires
    SET ACRES_FLOAT = CAST(ACRES AS FLOAT)
''')

In [22]:
con.sql('''
    DESCRIBE fires
''')

┌───────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│  column_name  │ column_type │  null   │   key   │ default │ extra │
│    varchar    │   varchar   │ varchar │ varchar │ varchar │ int32 │
├───────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ UNIT          │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ YEAR          │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ DECADE        │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ ACRES         │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ AGENCY        │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ NAME          │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ START         │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ CONTAINED     │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ DAYS          │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ CAUSE         │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ OBJECTID      │ VA

In [23]:
con.sql('''
    SELECT DECADE, SUM(CAST(ACRES AS FLOAT)) as Total_Acres
    FROM fires
    GROUP BY DECADE
    ORDER BY Total_Acres
''')

┌─────────┬────────────────────┐
│ DECADE  │    Total_Acres     │
│ varchar │       double       │
├─────────┼────────────────────┤
│ 1960    │ 2075062.6210534275 │
│ 1970    │ 2606294.3197754323 │
│ 1950    │ 2844981.3255828023 │
│ 1980    │ 3080518.3912314475 │
│ 1990    │ 3348844.0855171345 │
│ 2000    │  6529025.611988053 │
│ 2010    │  6898947.842089854 │
│ 2020    │  7010746.721358992 │
│         │ 8300465.1086936295 │
└─────────┴────────────────────┘

In [24]:
con.sql('''
    CREATE TABLE IF NOT EXISTS fires2 as
    SELECT * FROM ST_Read('cafires2.csv')    
''')
con.table('fires2')

┌─────────┬─────────┬─────────┬─────────┬─────────┬───┬─────────┬─────────┬──────────┬─────────────┬───────────────┐
│  UNIT   │  YEAR   │ DECADE  │  ACRES  │ AGENCY  │ … │  DAYS   │  CAUSE  │ OBJECTID │ Shape__Area │ Shape__Length │
│ varchar │ varchar │ varchar │ varchar │ varchar │   │ varchar │ varchar │ varchar  │   varchar   │    varchar    │
├─────────┼─────────┼─────────┼─────────┼─────────┼───┼─────────┼─────────┼──────────┼─────────────┼───────────────┤
│ NEU     │ 2020    │ 2020    │ 109.6   │ CDF     │ … │ 5       │ 11      │ 21440    │ 733134.7305 │ 4179.743142   │
│ NEU     │ 2020    │ 2020    │ 685.59  │ CDF     │ … │ 3       │ 2       │ 21441    │ 4578171.914 │ 12399.37539   │
│ NEU     │ 2020    │ 2020    │ 27.3    │ CDF     │ … │ 1       │ 14      │ 21442    │ 182387.6445 │ 2119.19412    │
│ NEU     │ 2020    │ 2020    │ 12.93   │ CDF     │ … │ 1       │ 9       │ 21443    │ 86679.44531 │ 2029.524881   │
│ NEU     │ 2020    │ 2020    │ 10.32   │ CDF     │ … │ 5       

In [47]:
con.sql('''
    SELECT DECADE, ROUND(SUM(CAST(ACRES AS FLOAT)),0) as Total_Acres
    FROM fires2
    GROUP BY DECADE
    ORDER BY Total_Acres DESC
    LIMIT 10
''')

┌─────────┬─────────────┐
│ DECADE  │ Total_Acres │
│ varchar │   double    │
├─────────┼─────────────┤
│ 2020    │   7010747.0 │
│ 2010    │   6898948.0 │
│ 2000    │   6529026.0 │
│ 1990    │   3348844.0 │
│ 1980    │   3080518.0 │
│ 1920    │   2925772.0 │
│ 1950    │   2844981.0 │
│ 1970    │   2606294.0 │
│ 1960    │   2075063.0 │
│ 1940    │   1959130.0 │
├─────────┴─────────────┤
│ 10 rows     2 columns │
└───────────────────────┘

In [48]:
con.sql('''
    SELECT YEAR, ROUND(SUM(CAST(ACRES AS FLOAT)),0) as Total_Acres
    FROM fires2
    GROUP BY YEAR
    ORDER BY Total_Acres DESC
    LIMIT 10
''')

┌─────────┬─────────────┐
│  YEAR   │ Total_Acres │
│ varchar │   double    │
├─────────┼─────────────┤
│ 2020    │   4181573.0 │
│ 2021    │   2512512.0 │
│ 2018    │   1581917.0 │
│ 2017    │   1424583.0 │
│ 2008    │   1382710.0 │
│ 2007    │   1031803.0 │
│ 2003    │    968697.0 │
│ 2002    │    963899.0 │
│ 1987    │    851816.0 │
│ 2012    │    848692.0 │
├─────────┴─────────────┤
│ 10 rows     2 columns │
└───────────────────────┘

In [43]:
con.sql('''
    SELECT CAST(YEAR AS INT) as YEAR, CAST(ACRES AS FLOAT) as ACRES, NAME, UNIT
    FROM fires2
    WHERE ACRES > 5000 AND YEAR >= 1970
    ORDER BY ACRES DESC
    LIMIT 15    
''')

┌───────┬────────────┬────────────────┬─────────┐
│ YEAR  │   ACRES    │      NAME      │  UNIT   │
│ int32 │   float    │    varchar     │ varchar │
├───────┼────────────┼────────────────┼─────────┤
│  2020 │  1032699.6 │ AUGUST COMPLEX │ MNF     │
│  2021 │  963405.44 │ DIXIE          │ BTU     │
│  2002 │  501082.03 │ BISCUIT        │ SIF     │
│  2018 │  410202.47 │ RANCH          │ LNU     │
│  2020 │  396824.47 │ SCU COMPLEX    │ SCU     │
│  2020 │   379842.4 │ CREEK          │ SNF     │
│  2020 │  318797.25 │ NORTH COMPLEX  │ PNF     │
│  2012 │   315511.5 │ RUSH           │ NOD     │
│  2020 │ 305351.875 │ HENNESSEY      │ LNU     │
│  2017 │ 281790.875 │ THOMAS         │ VNC     │
│  2003 │   270686.0 │ CEDAR          │ CNF     │
│  2013 │  256175.53 │ RIM            │ STF     │
│  2007 │   240358.7 │ ZACA           │ SBC     │
│  2018 │   229651.4 │ CARR           │ SHU     │
│  2021 │  223107.89 │ MONUMENT       │ SHF     │
├───────┴────────────┴────────────────┴─────────┤


In [50]:
df = pd.read_sql_query("SELECT * FROM fires2", con)
df

,UNIT,YEAR,DECADE,ACRES,AGENCY,NAME,START,CONTAINED,DAYS,CAUSE,OBJECTID,Shape__Area,Shape__Length
0,NEU,2020,2020,109.6,CDF,NELSON,18/06/2020,23/06/2020,5,11,21440,733134.7305,4179.743142
1,NEU,2020,2020,685.59,CDF,AMORUSO,01/06/2020,04/06/2020,3,2,21441,4578171.914,12399.37539
2,NEU,2020,2020,27.3,CDF,ATHENS,10/08/2020,11/08/2020,1,14,21442,182387.6445,2119.19412
3,NEU,2020,2020,12.93,CDF,FLEMING,31/03/2020,01/04/2020,1,9,21443,86679.44531,2029.524881
4,NEU,2020,2020,10.32,CDF,MELANESE,14/04/2020,19/04/2020,5,18,21444,70179.13281,1342.742903
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21844,VNC,2022,2020,1.55,LRA,CHAMBERS,16/03/2022,17/03/2022,1,14,43907,9217.691406,537.6913468
21845,VNC,2022,2020,1.42,LRA,MOON,14/08/2022,14/08/2022,0,14,43908,8444.238281,452.7993093
21846,SHF,2022,2020,12.91,USF,TEXAS,09/09/2022,10/09/2022,1,14,43909,89850.25781,2061.55088
21847,KNF,2022,2020,150.4,USF,ALEX,31/07/2022,03/08/2022,3,14,43910,1100802.367,6500.414625
